<a href="https://colab.research.google.com/github/lorenamaiorca/inteligenciaArtAva2023/blob/main/Entrega_3_IA2_G2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [18]:
#@title Acceder al Drive

# Nota: la primera vez se debe confirmar el uso logueandose en "Google Drive File Stream" y obteniendo código de autentificación.
from google.colab import drive
drive.mount('/content/gdrive')

# directorio local en Google Drive
path = '/content/gdrive/My Drive/Entrega3/'  #@param {type:"string"}


#@title Cargar datos


#@markdown ### Archivo de datos a utilizar:
archivo_datos = 'Entrega3Data.csv'  #@param {type:"string"}
#@markdown ### Configuración del archivo CSV:
delimitador_columnas = '|' #@param {type:"string"}

dataset = pd.read_csv(path+archivo_datos , delimitador_columnas)

dataset_predicciones = dataset.iloc[-6:].copy()
dataset = dataset.iloc[0:-6].copy()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-18-f6ad20cd4ecb>:19: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  dataset = pd.read_csv(path+archivo_datos , delimitador_columnas)


In [19]:
dataset_predicciones.shape

(6, 5)

In [20]:
dataset.shape

(72, 5)

In [21]:
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

print (X_test.shape)


(18, 3)


In [22]:
X.shape

(72, 3)

In [23]:
y.shape

(72,)

In [24]:

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

sc2 = StandardScaler()
y_train = sc2.fit_transform(y_train.reshape(-1,1))
y_test = sc2.transform(y_test.reshape(-1,1))

media_y = y.mean()
desvio_estandar_y = y.std()

In [25]:
X_train.shape

(54, 3)

In [26]:
y_train.shape

(54, 1)

In [27]:
#DADO que LSTM requiere input ne 3 dimensiones, y que nosotros tenemos 2 dimensiones (matrices), haces un reshape

X_train = X_train.reshape(-1, 1, X_train.shape[1])
X_test = X_test.reshape(-1, 1, X_test.shape[1])

y_train = y_train.reshape(-1, 1, y_train.shape[1])

y_test = y_test.reshape(-1, 1, y_test.shape[1])

In [28]:
y_train.shape

(54, 1, 1)

In [29]:
X_train.shape

(54, 1, 3)

In [30]:
y_test.shape

(18, 1, 1)

In [31]:
X_test.shape

(18, 1, 3)

In [33]:
red = tf.keras.models.Sequential()
red.add(tf.keras.layers.LSTM(units=9, activation='relu'))
red.add(tf.keras.layers.Dense(units=9, activation='relu'))
#red.add(tf.keras.layers.Dense(units=6, activation='relu'))
red.add(tf.keras.layers.Dense(units=1, activation='linear'))
red.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.05),
            loss = 'mse',
            metrics = ['mse', 'mae'])

# ENTRENAMOS
red.fit(X_train, y_train, epochs = 5000)

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 10ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0428
Epoch 1184/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0430
Epoch 1185/5000
2/2 [==============================] - 0s 12ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0431
Epoch 1186/5000
2/2 [==============================] - 0s 12ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0432
Epoch 1187/5000
2/2 [==============================] - 0s 10ms/step - loss: 0.0033 - mse: 0.0033 - mae: 0.0430
Epoch 1188/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.0032 - mse: 0.0032 - mae: 0.0433
Epoch 1189/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0425
Epoch 1190/5000
2/2 [==============================] - 0s 11ms/step - loss: 0.0031 - mse: 0.0031 - mae: 0.0428
Epoch 1191/5000
2/2 [==============================] - 0s 9ms/step - loss: 0.003

In [ ]:
y_pred = red.predict(X_test)

for i in range(len(y_pred)):
    error_absoluto = abs(y_test[i][0][0] - y_pred[i][0])
    error_relativo = abs(error_absoluto / y_test[i][0][0])*100

    line_info = f"ERROR RELATIVO % {error_relativo} || ERROR ABSOLUTO : {error_absoluto}"
    print(line_info)


In [ ]:
red.summary()

In [ ]:
# apreciera un modelo "bueno", probemos con los dataset de validaciones que sacamos al principio y
# tratemos de jugar para ver lo bien que funciona

X_para_predicciones = dataset_predicciones.iloc[:, 1:-1]
y_para_predicciones = dataset_predicciones.iloc[:, -1]


In [ ]:
X_para_predicciones

In [ ]:
y_para_predicciones

In [ ]:
# ARMAMOS LA FUNCION A MANO SIN NUMPY PARA QUE SE VEA BIEN LO QUE HACEMOS
# (ES PARECIDO A LO QUE HICIMOS CON EL TESTING SET MÁS ARRIBA )

print("IPC REAL | IPC MODELO | ERROR ABSOLUTO  (MÓDULO) | ERROR RELATIVO ")
for i in range(len(X_para_predicciones)):
    compras_super = X_para_predicciones.iloc[i, 0]
    compras_shopping = X_para_predicciones.iloc[i, 1]
    nafta_y_gasoil = X_para_predicciones.iloc[i, 2]
    IPC = y_para_predicciones.iloc[i]

    valor_de_prueba = sc.transform([[ compras_super , compras_shopping , nafta_y_gasoil ]])
    valor_de_prueba = valor_de_prueba.reshape(-1, 1, valor_de_prueba.shape[1])

    predecido = red.predict( valor_de_prueba )[0][0]

    prediccion_numerica = predecido*desvio_estandar_y + media_y

    error_absoluto = abs(IPC - prediccion_numerica)

    error_relativo = abs(error_absoluto / IPC)

    line_info = f"{IPC} | {prediccion_numerica} | {error_absoluto} | {error_relativo}"

    print(line_info)